In [1]:
%load_ext autoreload

In [2]:
%autoreload
import numpy as np
import darkhistory.spec.transferfunction as tf
from darkhistory.spec.spectrum import Spectrum
import pickle
import darkhistory.utilities as utils

In [4]:
file_name = "/Users/hongwan/Dropbox (MIT)/Photon Deposition/transferfunction_withloweng_nointerp.npy"

In [5]:
test = np.load(file_name)

In [6]:
test.shape

(500, 50, 1, 500, 3)